## Saving Models in Deep Learning

---

In [1]:
#Import Dependencies
import warnings
warnings.simplefilter('ignore', FutureWarning)

import pandas as pd
import numpy as np

## Data Pre-Processing

In [2]:
voice = pd.read_csv('../../../dataset/voice.csv')
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [3]:
#Assign data and target

X = voice.drop('label', axis=1)
y = voice['label']
print(X.shape,y.shape)

(3168, 20) (3168,)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical

Using TensorFlow backend.


In [5]:
#Split the data into train and test

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

#Scale the data
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [6]:
#Step 1: Label encode the target set
label_encoder = LabelEncoder().fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [7]:
#Step 2: convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

## Create a Deep Learning Model

In [8]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=20))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [9]:
model.compile(optimizer='adam',
             loss ='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train_scaled, y_train_categorical, epochs=60, shuffle=True, verbose=0)

## Quantify our Trained Model

In [10]:
model_loss,  model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)

print(f'Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}')

Normal Neural Network - Loss: 0.04566098877982005, Accuracy: 0.9911616161616161


# Saving a Trained Model
We can save our trained models using the HDF5 binary format with the extension `.h5`

In [11]:
#Save the model

model.save('../../../models/voice_model_trained.h5')

## Loading a model

In [12]:
#Load the model

from keras.models import load_model
voice_model = load_model('../../../models/voice_model_trained.h5')

## Evaluating the Loaded Model

In [13]:
model_loss, model_accuracy = voice_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)

print(f'Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}')

Normal Neural Network - Loss: 0.04566098877982005, Accuracy: 0.9911616161616161
